# Sales Transaction Report - Transformation

In [1]:
import pandas as pd
import numpy as np

## Custom Function

In [2]:
def convert_object_to_cat_type(df, columns):
  """
  Convert Object to Category datatype in pandas dataframe
  """
  for col in columns:
    df[col] = df[col].astype('category')

In [3]:
def convert_dtype_to_float_type(df, columns):
  """
  Convert object to float datatype in pandas dataframe
  """
  for col in columns:
    df[col] = df[col].astype(float)

In [4]:
def convert_dtype_to_boolean_type(df, columns):
  """
  Convert object to boolean datatype in pandas dataframe
  """
  for col in columns:
    df[col] = df[col].astype(bool)

In [5]:
def map_boolean(df, columns):
    """
    map yes/no to 1/0
    """
    for col in columns:
        df[col] = df[col].map(dict(Yes = 1, No = 0))

In [6]:
def convert_custom_date(df, columns):
    """
    input -  dd/mm/yyyy
    output - dd-mm-yyyy
    since already format of the date is corrent,
    we will change '/' to '-'
    """
    for col in columns:
        df[col] = df[col].str.replace('/', '-', regex = True)

In [7]:
def add_start_month(df, column_name, new_column_name):
    """
    add month_start column based on the sales transaction date
    """
    df[new_column_name] = df[column_name].apply(lambda x: '01' + x[2:10])
        
    return df 

In [8]:
def check_quarantine_condition(df):
    # Irish county list
    irish_counties_list = ['galway', 'leitrim', 'mayo', 'roscommon', 'sligo', 'carlow', 'dublin', 'kildare', 'kilkenny', 'laois', 'longford', 'louth', 'meath', 'offaly', 'westmeath', 'wexford', 'wicklow', 'clare', 'cork', 'kerry',
        'limerick', 'tipperary', 'waterford', 'cavan', 'donegal', 'monaghan', 'antrim', 'armagh', 'down', 'fermanagh', 'londonderry', 'tyrone']
    df['quarantine_ind'] = df.county.str.lower().apply(lambda x: 0 if x in irish_counties_list else 1)
    df['quarantine_code'] = df.quarantine_ind.apply(lambda x: "NOT IRISH COUNTIES" if x == 1 else "")
    return df

## Read Data

In [9]:
# https://stackoverflow.com/a/18172249 - encoding reference
df = pd.read_csv('PPR-ALL.csv', encoding = "ISO-8859-1", names=['sales_date', 'address', 'postal_code', 'county', 'sales_value', 'not_full_market_price_ind', 'vat_exclusion_ind', 'property_desc', 'property_size_desc' ], nrows=20, skiprows=1)

## Preprocessing data

In [10]:
# remove unused columns
df.drop(['address', 'postal_code', 'property_size_desc'], axis = 1, inplace = True)

In [11]:
# remove starting special character and comma 
df.sales_value = df.sales_value.str.replace('[^\d.]', '', regex = True)

In [12]:
# convert object to float type
convert_dtype_to_float_type(df, ['sales_value'])

In [13]:
# map YES/NO to 1/0 type
map_boolean(df, ['not_full_market_price_ind', 'vat_exclusion_ind'])

In [14]:
# convert date to mm-dd-yyyy standard
# function help us to handle the convertion as required
convert_custom_date(df, ['sales_date'])

In [15]:
# convert object to category type
convert_object_to_cat_type(df, ['county'])

In [16]:
# map new/second-hand properties to 1/0
df['new_home_ind'] = df['property_desc'].map({'New Dwelling house /Apartment' : 1, 'Second-Hand Dwelling house /Apartment' : 0})

In [17]:
## add_start_month
df = add_start_month(df, 'sales_date', 'month_start')

In [20]:
df = check_quarantine_condition(df)

In [21]:
df.dtypes

sales_date                     object
county                       category
sales_value                   float64
not_full_market_price_ind       int64
vat_exclusion_ind               int64
property_desc                  object
new_home_ind                    int64
month_start                    object
quarantine_ind                  int64
quarantine_code                object
dtype: object

In [19]:
# df.county.str.lower().apply(lambda x: 0 if x in irish_counties_list else 1)

NameError: name 'irish_counties_list' is not defined

In [27]:
df.loc[(df['not_full_market_price_ind'] == 1) & (df['vat_exclusion_ind'] == 1) & (df['new_home_ind'] == 0), ['quarantine_ind', 'quarantine_code']] = [1, 'NEW HOME NOT FULL MARKET VALUE']

,sales_date
